In [1]:
from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
import time

robot = Robot()

In [2]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

In [3]:
import zmq
import json
import time
from jetbot import Robot
robot = Robot()
class Kinematics:
    """Handle robot kinematics calculations"""
    
    def __init__(self, wheel_radius, wheel_distance, pulses_per_turn):
        self.wheel_radius = wheel_radius
        self.wheel_distance = wheel_distance
        self.pulses_per_turn = pulses_per_turn
    
    def wheel_speed_commands(self, u_desired, w_desired):
        """Converts desired speeds to wheel speed commands
        
        Args:
            u_desired: Desired linear speed [m/s]
            w_desired: Desired angular speed [rad/s]
            
        Returns:
            Tuple of (wl_desired, wr_desired) wheel speeds [rad/s]
        """
        wr_desired = float((2*u_desired + self.wheel_distance*w_desired) / (2*self.wheel_radius))
        wl_desired = float((2*u_desired - self.wheel_distance*w_desired) / (2*self.wheel_radius))
        return wl_desired, wr_desired
WHEEL_RADIUS = 0.325
WHEEL_DISTANCE = 0.15
PULSES_PER_TURN = 330

kinematics = Kinematics(WHEEL_RADIUS, WHEEL_DISTANCE, PULSES_PER_TURN)   
left_speed, right_speed = kinematics.wheel_speed_commands(0, 0)
robot.set_motors(left_speed, right_speed)


In [ ]:
import zmq
import time
import cv2
import json

from jetbot import Camera, Robot

# ----------------------------
# CONFIG
# ----------------------------
WINDOWS_IP = "192.168.0.101"

VIDEO_PORT = 5555
CTRL_PORT  = 5556

QUALITY = 70
TARGET_FPS = 20
DT = 1.0 / TARGET_FPS

# If your kinematics is different, replace this mapping:
def linear_angular_to_motors(linear, angular):
    left  = linear - angular
    right = linear + angular
    # clamp [-1, 1]
    left  = max(-1.0, min(1.0, left))
    right = max(-1.0, min(1.0, right))
    return left, right

# ----------------------------
# ZMQ CONTEXT
# ----------------------------
ctx = zmq.Context()

# Video publisher (Jetson -> Windows)
pub = ctx.socket(zmq.PUB)
pub.setsockopt(zmq.CONFLATE, 1)
pub.setsockopt(zmq.SNDHWM, 1)
pub.connect(f"tcp://{WINDOWS_IP}:{VIDEO_PORT}")

# Control subscriber (Windows -> Jetson)
sub = ctx.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")
sub.setsockopt(zmq.RCVTIMEO, 50)   # short timeout keeps loop responsive
sub.setsockopt(zmq.CONFLATE, 1)
sub.setsockopt(zmq.RCVHWM, 1)
sub.connect(f"tcp://{WINDOWS_IP}:{CTRL_PORT}")

# ----------------------------
# ROBOT + CAMERA
# ----------------------------
robot = Robot()
camera = Camera.instance(width=160, height=160)

time.sleep(0.5)  # allow sockets to connect (slow joiner)

last_cmd_t = time.time()
last_fps_t = time.time()
frames = 0

print("Started: streaming video + receiving control")

try:
    while True:
        t0 = time.time()

        # -------- VIDEO SEND --------
        frame = camera.value
        if frame is not None:
            ok, enc = cv2.imencode(".jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, QUALITY])
            if ok:
                pub.send(enc.tobytes())
                frames += 1

        # -------- CONTROL RECEIVE (non-blocking via timeout) --------
        try:
            msg = sub.recv_string()      # raises zmq.Again on timeout
            data = json.loads(msg)
            linear  = float(data.get("linear", 0.0))
            angular = float(data.get("angular", 0.0))

            left, right = linear_angular_to_motors(linear, angular)
            robot.set_motors(left, right)
            last_cmd_t = time.time()

            # optional debug
            # print(f"linear={linear:.2f} angular={angular:.2f} -> L={left:.2f} R={right:.2f}")

        except zmq.Again:
            # No new command this loop
            pass

        # -------- SAFETY STOP --------
        if time.time() - last_cmd_t > 0.5:
            robot.stop()

        # -------- FPS PRINT --------
        if time.time() - last_fps_t >= 1.0:
            print("fps:", frames)
            frames = 0
            last_fps_t = time.time()

        # -------- RATE LIMIT --------
        sleep = DT - (time.time() - t0)
        if sleep > 0:
            time.sleep(sleep)

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    robot.stop()
    pub.close()
    sub.close()
    ctx.term()


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetbot-0.4.3-py3.6.egg/jetbot/heartbeat.py", line 35, in _run
    self.status = Heartbeat.Status.dead
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py",

Started: streaming video + receiving control
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 18
fps: 19
fps: 19
fps: 19
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 19
fps: 19
fps: 20
fps: 19
fps: 19
fps: 20
fps: 19
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 19
fps: 18
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 18
fps: 19
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 20
fps: 19
fps: 19
fps: 19
fps: 20
fps: 20
fps: 20
fps: 20
fps: 19
fps: 20
fps: 20
fps: 21
fps: 19


In [ ]:
try:
    # your loop
    ...
except OSError as e:
    print("Motor I2C error:", e)
finally:
    try:
        robot.stop()
    except OSError as e:
        print("Could not stop motors due to I2C error:", e)

In [ ]:
import zmq, time, cv2
from jetbot import Camera

WINDOWS_IP = "192.168.0.101"
PORT = 5555
QUALITY = 70
TARGET_FPS = 20

ctx = zmq.Context()
pub = ctx.socket(zmq.PUB)
pub.setsockopt(zmq.CONFLATE, 1)
pub.setsockopt(zmq.SNDHWM, 1)
pub.connect(f"tcp://{WINDOWS_IP}:{PORT}")

CTRL_PORT = 5556

robot = Robot()

ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")
sub.setsockopt(zmq.RCVTIMEO, 200)   # 200ms timeout
sub.connect(f"tcp://{WINDOWS_IP}:{CTRL_PORT}")

last_cmd_t = time.time()
camera = Camera.instance(width=160, height=160)
time.sleep(0.5)

dt = 1.0 / TARGET_FPS
last = time.time()
frames = 0

while True:
     try:
        t0 = time.time()
        frame = camera.value
        ok, enc = cv2.imencode(".jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, QUALITY])
        if ok:
            pub.send(enc.tobytes())
        if sleep > 0:
            time.sleep(sleep)
            msg = sub.recv_string()
            data = json.loads(msg)
            linear  = data.get("linear", 0.0)
            angular = data.get("angular", 0.0)

            print(f" linear={linear:.3f} | angular={angular:.3f}")
            left_speed, right_speed = kinematics.wheel_speed_commands(linear, angular)
            robot.set_motors(left_speed, right_speed)
            last_cmd_t = time.time()
        # fps print once/sec
        frames += 1
        if time.time() - last >= 1.0:
            print("fps:", frames)
            frames = 0
            last = time.time()

            sleep = dt - (time.time() - t0)


        except zmq.Again:
            # safety stop if no commands recently
            if time.time() - last_cmd_t > 0.5:
                robot.stop()    